Given n men and n women, where each person has ranked all potential mates (from the opposite sex), match those people so that no two people of opposite sex would both rather have each other than their current partners.

In [104]:
import pandas as pd
df_people = pd.read_excel("Preference List.xlsx")
type(df_people.loc[0, "Preferences"])
df_people

,Name,Gender,Preferences,Partner
0,Anne,f,"Dieter, Emil, Claudio, Berthold, Anton",NaN
1,Berta,f,"Dieter, Emil, Claudio, Berthold, Anton",NaN
2,Claudia,f,"Berthold, Anton, Dieter, Emil, Claudio",NaN
3,Diana,f,"Claudio, Berthold, Anton, Dieter, Emil",NaN
4,Fiona,f,"Dieter, Emil, Claudio, Berthold, Anton",NaN
5,Anton,m,"Claudia, Fiona, Diana, Berta, Anne",NaN
6,Berthold,m,"Claudia, Fiona, Diana, Berta, Anne",NaN
7,Claudio,m,"Berta, Anne, Claudia, Fiona, Diana",NaN
8,Dieter,m,"Diana, Berta, Anne, Claudia, Fiona",NaN
9,Emil,m,"Fiona, Diana, Berta, Anne, Claudia",NaN


In [105]:
df_people["Preferences"].str.split(",", expand=False)[0]

['Dieter', ' Emil', ' Claudio', ' Berthold', ' Anton']

In [106]:
df_people["Preferences"] = df_people["Preferences"].str.split(", ", expand=False)
df_people["Partner"] = None
df_people

,Name,Gender,Preferences,Partner
0,Anne,f,"[Dieter, Emil, Claudio, Berthold, Anton]",None
1,Berta,f,"[Dieter, Emil, Claudio, Berthold, Anton]",None
2,Claudia,f,"[Berthold, Anton, Dieter, Emil, Claudio]",None
3,Diana,f,"[Claudio, Berthold, Anton, Dieter, Emil]",None
4,Fiona,f,"[Dieter, Emil, Claudio, Berthold, Anton]",None
5,Anton,m,"[Claudia, Fiona, Diana, Berta, Anne]",None
6,Berthold,m,"[Claudia, Fiona, Diana, Berta, Anne]",None
7,Claudio,m,"[Berta, Anne, Claudia, Fiona, Diana]",None
8,Dieter,m,"[Diana, Berta, Anne, Claudia, Fiona]",None
9,Emil,m,"[Fiona, Diana, Berta, Anne, Claudia]",None


## Implementation of the Galey-Shapley Algorithm

Every free men gets a women according to his preference (if it is still free). Assign women to men by iterating over all men.

If a woman is not free, and she finds that she linked the proposing man more than her current partner, then she takes the proposing man.

Repeat the entire process over and over until everybody is married

In [ ]:
#def validate_program_state(df_people):
#    count_number_of_partners = 0
#    for index, row in df_people.iterrows():
#        if row["Partner"] is not None:
#            count_number_of_partners = count_number_of_partners + 1
#        
#    if df_people["Partner"]:
#        raise ValueError("Error occurred : -(")
        

In [108]:
df_people = df_people.set_index("Name", drop=False) #if don't input drop = False, it will not create a new index, but move the original one to the most left to work as the index

while pd.isnull(df_people["Partner"]).any():
    for man_index, man_row in df_people[(df_people["Gender"] == "m") & (df_people["Partner"].isnull())].iterrows(): #itterrows的格式：for index, row in df.iterrows(), 其结果是在dataframe中的行进行迭代，返回每行的索引和一个包含行的本身的对象
        #print(f"\nPreferences for {man_index}: {man_row['Preferences']}")

        for woman_index in man_row["Preferences"]:
            current_partner_of_the_woman = df_people.loc[woman_index, "Partner"]
    #       print("\n\n\n")
    #       print(f"Man:{man_index}\n{df")

            if df_people.loc[woman_index, "Partner"] == None: #when woman in the preference of the 1st man(outcome of the iterrows) is free
                df_people.loc[woman_index, "Partner"] = man_index #assign this woman to the 1st man
                df_people.loc[man_index, "Partner"] = woman_index #assign the woman in the previous line to the man
        
                break # Continue -> Immediately jump to the next element in the current loop. 
        
        
            else: #if a woman is not free...
                propossing_man = man_index #上一步中针对的man            
                woman_preferences = df_people.loc[woman_index, 'Preferences']
                if woman_preferences.index(propossing_man) < woman_preferences.index(current_partner_of_the_woman):
                    #divorce and new partner
                    #change current partner of the woman to the proposing man
                    #existing prior partner is now alone
                    #new partner get married to the woman
                    df_people.loc[woman_index, "Partner"] = propossing_man
                    df_people.loc[current_partner_of_the_woman, "Partner"] = None 
                    df_people.loc[man_index, "Partner"] = woman_index #man_index 已经是propossing man

                    break

print(df_people)


              Name Gender                               Preferences   Partner
Name                                                                         
Anne          Anne      f  [Dieter, Emil, Claudio, Berthold, Anton]   Claudio
Berta        Berta      f  [Dieter, Emil, Claudio, Berthold, Anton]    Dieter
Claudia    Claudia      f  [Berthold, Anton, Dieter, Emil, Claudio]  Berthold
Diana        Diana      f  [Claudio, Berthold, Anton, Dieter, Emil]     Anton
Fiona        Fiona      f  [Dieter, Emil, Claudio, Berthold, Anton]      Emil
Anton        Anton      m      [Claudia, Fiona, Diana, Berta, Anne]     Diana
Berthold  Berthold      m      [Claudia, Fiona, Diana, Berta, Anne]   Claudia
Claudio    Claudio      m      [Berta, Anne, Claudia, Fiona, Diana]      Anne
Dieter      Dieter      m      [Diana, Berta, Anne, Claudia, Fiona]     Berta
Emil          Emil      m      [Fiona, Diana, Berta, Anne, Claudia]     Fiona
